template matching

In [31]:
import cv2
import pytesseract
import numpy as np
import pandas as pd

text = pytesseract.image_to_string('name.jpg', lang='kor')

text_list = text.split('\n')
print(text_list)
for text in text_list:
    if len(text)<3:
        text_list.remove(text)
        
text_dict = {'name':'name', 'address':'address'}
for text in text_list:
    if 2<=len(text)<=6:
        text_dict['name'] = text
    elif len(text) >=8:
        if len(text_dict['address']) < len(text):
            text_dict['address'] = text
    
    print(text_dict)

['그린배터리', '상최 그 린', '', '40606. 010.0000.0000:', '6-메에, 0100000000000.000', '인천광역시 남동구 구월동 1451번지 정구프라자 1층', '\x0c']
{'name': '그린배터리', 'address': 'address'}
{'name': '상최 그 린', 'address': 'address'}
{'name': '상최 그 린', 'address': '40606. 010.0000.0000:'}
{'name': '상최 그 린', 'address': '6-메에, 0100000000000.000'}
{'name': '상최 그 린', 'address': '인천광역시 남동구 구월동 1451번지 정구프라자 1층'}


## Template matching

In [1]:
#match templete
import cv2
import numpy as np
import sys

lenna = cv2.imread('lenna.jpg')
template = cv2.imread('face.jpg')

if lenna is None or template is None:
    print('Image lead failed')

lenna = lenna + (50, 50, 50) #밝기 50 증가

#가우시안잡음 추가
noise = np.zeros(lenna.shape, np.int32)
cv2.randn(noise, 0, 10)
lenna = cv2.add(lenna, noise, dtype=cv2.CV_8UC3)

lenna2 = lenna.copy()

match = cv2.matchTemplate(lenna, template, cv2.TM_CCORR_NORMED)

_, maxv, _, maxloc = cv2.minMaxLoc(match)

th, tw = template.shape[:2]

cv2.rectangle(lenna2, maxloc, (maxloc[0]+tw, maxloc[1]+th), (0,0,255), 2)

merge = np.hstack((lenna,lenna2))

cv2.imshow('match', merge)

cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
import numpy as np
import cv2

obj = cv2.imread('./images/spades.png', cv2.IMREAD_GRAYSCALE)
src = cv2.imread('./images/symbols.png', cv2.IMREAD_GRAYSCALE)

if obj is None or src is None:
    print('Image lead failed')
    
# 객체 영상 외곽선 검출
_, obj_bin = cv2.threshold(obj, 128, 255, cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
obj_pts = obj_contours[0]

#입력 영상 분석
_, src_bin = cv2.threshold(src, 128, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE )

#결과영상
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

# 입력영상 모든 객체 영역 대해
for pts in contours :
    if cv2.contourArea(pts) < 1000:
        continue
        
    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255, 0, 0), 1)
    
    #비교
    dist= cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)
    
    cv2.putText(dst, str(round(dist,4)), (rc[0], rc[1] - 3),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0))
    
    if dist < 0.1:
        cv2.rectangle(dst, rc, (0,0,255), 2)

cv2.imshow('obj', obj)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
#template match
import sys
import numpy as np
import cv2

# 입력영상, 템플릿 영상 불러오기
src = cv2.imread('./images/template_matching_ex1.png', cv2.IMREAD_GRAYSCALE)
templ = cv2.imread('./images/template_matching_ex2.png', cv2.IMREAD_GRAYSCALE)

if src is None or templ is None:
    print('Image load failed')
    
# 입력영상 밝기 50 증가 and 가우시안 잡음 추가
noise = np.zeros(src.shape, np.int32)
cv2.randn(noise, 50, 10)
src = cv2.add(src, noise, dtype=cv2.CV_8UC3)

# template matching,결과분석
res = cv2.matchTemplate(src, templ, cv2.TM_CCOEFF_NORMED)

#res=TM_COEFF_NORMED 의 결과이므로 -1~1값 가짐
# grayscale(0~255)로 나타내기 위해 normalize
res_norm = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# MinMaxLoc 결과 : min값,  max값, min위치, max위치
_, maxv, _, maxloc = cv2.minMaxLoc(res)

print('maxv:',maxv)
print('maxloc:', maxloc)

#매칭 결과 빨간색 표시
th, tw = templ.shape[:2]
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.rectangle(dst, maxloc, (maxloc[0]+tw, maxloc[1]+th), (0,0,255), 2)

cv2.imshow('res_norm', res_norm)
cv2.imshow('template', templ)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

maxv: 0.6846160888671875
maxloc: (330, 219)
